# 🚀 Day 2 실습 4: 고급 쿼리 엔지니어링

**목표**: 복잡한 질문을 RAG가 이해하기 쉽게 변환하는 3가지 기법 실습

**소요 시간**: 60분

---

## 📚 배울 내용

1. **Multi-Query**: 복합 질문 → 여러 개 단순 질문으로 분해
2. **HyDE**: 질문 → 가상의 답변 생성 → 답변으로 검색
3. **Step-Back**: 구체적 질문 → 추상적 질문으로 변환

---

## ✨ 실습 방식

- 각 기법마다 **거의 완성된 코드** 제공
- 여러분은 **프롬프트 1-2줄만 수정**하면 됩니다!
- 각 케이스마다 관련 문서를 명확히 보여드립니다

In [ ]:
# 라이브러리 설치
!pip install -q langchain-community sentence-transformers faiss-cpu langchain-huggingface

In [ ]:
# 공통 설정
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Embedding 모델 로딩
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
print("✅ Embedding 모델 로딩 완료")

In [ ]:
# LLM 로딩 (EXAONE 2.4B)
model_id = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.7
)

llm = HuggingFacePipeline(pipeline=pipe)
print("✅ LLM 로딩 완료")

In [ ]:
# 헬퍼 함수
import re

def parse_queries(response, max_queries=3):
    """LLM 응답에서 쿼리 추출"""
    # 쉼표 또는 줄바꿈으로 split
    if ',' in response:
        queries = [q.strip() for q in response.split(',') if q.strip()]
    else:
        queries = [q.strip() for q in response.split('\n') if q.strip() and len(q.strip()) > 10]
    
    # 숫자 제거
    queries = [re.sub(r'^[\d\.\)]+\s*', '', q) for q in queries][:max_queries]
    return queries

def check_result(found_types, needed_types):
    """검색 결과 확인"""
    found = sum(1 for t in needed_types if t in found_types)
    print(f"   검색된 문서: {found_types}")
    print(f"   성공률: {found}/{len(needed_types)}")
    return found

print("✅ 헬퍼 함수 준비 완료")

---

## 🔹 Part 1: Multi-Query - 복합 질문 분해 (20분)

### 📋 케이스: 주말 파스텔 여행 계획

**질문**: "주말에 파스텔에서 공연 보고 호텔 숙박하고 카페 체험하고 싶어요"

**목표**: 3가지 정보를 모두 찾아야 합니다
1. 🏨 **호텔** 정보
2. ☕ **카페** 정보
3. 🎭 **극장** 정보

**관련 문서**:

In [ ]:
# Multi-Query 케이스 - 파스텔 여행 (문서 10개)
mq_docs_raw = [
    # 정답 문서 3개 (파스텔만 포함)
    {"content": "파스텔 호텔: 서울 강남구 위치, 객실 50개, 주말 특가 이벤트 진행 중. 체크인 14시, 체크아웃 11시. 조식 포함.", "type": "hotel"},
    {"content": "파스텔 카페: 제주도 애월읍 위치, 오션뷰 좌석, 수제 디저트 특화. 영업시간 10-22시. 주차 가능.", "type": "cafe"},
    {"content": "파스텔 소극장: 부산 해운대 위치, 좌석 100석, 매주 토요일 저녁 8시 공연. 예매는 홈페이지에서.", "type": "theater"},
    # Noise 문서 7개 (파스텔 없음)
    {"content": "주말 강남 여행 가이드: 맛집 투어, 숙박 시설, 카페 거리 추천 코스.", "type": "blog"},
    {"content": "서울 문화생활 안내: 호텔, 카페, 소극장을 함께 즐기는 방법.", "type": "intro"},
    {"content": "주말 이벤트 특가: 공연 할인, 숙박 할인, 디저트 체험 행사.", "type": "event"},
    {"content": "제주도 애월 카페 거리: 오션뷰 카페 추천, 주차 정보.", "type": "travel"},
    {"content": "부산 해운대 공연 일정: 소극장 뮤지컬, 토요일 저녁 공연.", "type": "culture"},
    {"content": "호텔 예약 꿀팁: 주말 특가 찾기, 체크인 시간 확인 방법.", "type": "tips"},
    {"content": "카페 체험 프로그램: 수제 디저트 만들기, 예약 방법 안내.", "type": "program"},
]

mq_docs = [Document(page_content=d["content"], metadata={"type": d["type"]}) for d in mq_docs_raw]
mq_vectorstore = FAISS.from_documents(mq_docs, embeddings)

print(f"📚 Multi-Query 문서 {len(mq_docs_raw)}개 로딩:")
print("  정답 문서 3개: '파스텔' 포함 (hotel, cafe, theater)")
print("  Noise 문서 7개: '파스텔' 없음")

In [ ]:
# Multi-Query 데모 실행
mq_query = "주말에 파스텔에서 공연 보고 호텔 숙박하고 카페 체험하고 싶어요"

# 1) Naive RAG (비교용)
print("🔍 Naive RAG (원본 질문 그대로):")
naive_results = mq_vectorstore.similarity_search(mq_query, k=3)
naive_types = [r.metadata["type"] for r in naive_results]
check_result(naive_types, ["hotel", "cafe", "theater"])

print("\n" + "="*50)

# 2) Multi-Query (정답 예시)
print("\n🚀 Multi-Query (질문 분해):")

# 전문가가 분해한 예시
sub_queries = [
    "파스텔 호텔",
    "파스텔 카페",
    "파스텔 소극장"
]

print(f"   분해된 질문 {len(sub_queries)}개:")
for i, q in enumerate(sub_queries, 1):
    print(f"   {i}. {q}")

# 각 질문으로 검색 (k=1씩)
all_results = []
for q in sub_queries:
    results = mq_vectorstore.similarity_search(q, k=1)
    all_results.extend(results)

mq_types = [r.metadata["type"] for r in all_results]
print("\n📊 Multi-Query 결과:")
check_result(mq_types, ["hotel", "cafe", "theater"])

---

## 🔄 Part 1: Multi-Query (복합 질문 분해)

### 📌 언제 사용하나요?

복합적인 요구사항이 여러 개 섞여있을 때 유용합니다:

**✅ 사용하기 좋은 경우:**
- "호텔 숙박하고 카페 가고 공연 보고 싶어요" ← 3가지 독립적 요청
- "쇼핑하고 레스토랑 가고 놀이기구 타고 싶어요"
- "A도 궁금하고 B도 궁금하고 C도 알고 싶어요"

**❌ 효과가 적은 경우:**
- "파스텔 호텔 위치 알려줘" ← 단일 질문
- "체크인 시간이 언제인가요?" ← 구체적인 단일 질문

### 🎯 핵심 아이디어

1개의 복합 질문 → **여러 개의 간결한 검색어**로 분해
- "파스텔 공연, 파스텔 호텔, 파스텔 카페" 각각 검색
- 검색 결과를 합쳐서 더 풍부한 정보 제공

---

### 💡 실습 1: 복합 질문 분해하기

**미션**: 프롬프트를 작성해서 3/3 성공시키기!

**요구사항**: 복합 질문을 쉼표로 구분된 간결한 검색어로 분해하세요

In [ ]:
# 여기를 수정하세요! 👇
# TODO: Few-shot 프롬프트를 작성해서 복합 질문을 간결한 검색어로 분해하세요

my_query = "주말에 파스텔에서 공연 보고 호텔 숙박하고 카페 체험하고 싶어요"

# Few-shot 프롬프트 작성
prompt = """"""

# LLM 호출
response = llm.invoke(prompt.format(query=my_query))

# 응답 파싱
if "출력:" in response:
    last_output = response.split("출력:")[-1].strip()
    sub_queries = parse_queries(last_output)
else:
    sub_queries = parse_queries(response)

print(f"생성된 검색어: {sub_queries}")

# 생성된 검색어로 실제 검색
all_results = []
for sq in sub_queries:
    results = mq_vectorstore.similarity_search(sq, k=1)
    all_results.extend(results)

# 결과 확인
found_types = set(doc.metadata['type'] for doc in all_results)
target_types = {'theater', 'hotel', 'cafe'}
success = len(found_types & target_types)

print(f"검색된 문서: {list(found_types)}")
print(f"성공률: {success}/3 {'✅' if success >= 2 else '❌'}")

<details>
<summary>📝 정답 코드 보기 (클릭)</summary>

```python
# Few-shot 프롬프트
prompt = """당신은 복합 질문을 간결한 검색어로 분해하는 전문가입니다.

[예시 1]
입력: 롯데월드 놀이기구 타고 레스토랑 가고 싶어요
출력: 롯데월드 놀이기구, 롯데월드 레스토랑

[예시 2]
입력: 명동에서 쇼핑하고 호텔 묵고 공연 보고 싶어요
출력: 명동 쇼핑, 명동 호텔, 명동 공연

[실제 작업]
입력: {query}
출력:"""

# LLM 호출
response = llm.invoke(prompt.format(query=my_query))

# 응답 파싱
if "출력:" in response:
    last_output = response.split("출력:")[-1].strip()
    sub_queries = parse_queries(last_output)
else:
    sub_queries = parse_queries(response)

print(f"생성된 검색어: {sub_queries}")

# 생성된 검색어로 실제 검색
all_results = []
for sq in sub_queries:
    results = mq_vectorstore.similarity_search(sq, k=1)
    all_results.extend(results)

# 결과 확인
found_types = set(doc.metadata['type'] for doc in all_results)
target_types = {'theater', 'hotel', 'cafe'}
success = len(found_types & target_types)

print(f"검색된 문서: {list(found_types)}")
print(f"성공률: {success}/3 {'✅' if success >= 2 else '❌'}")
```

**핵심 포인트:**
- Few-shot 예시 2개로 출력 형식 학습
- 쉼표로 구분된 간결한 검색어 생성
- 각 검색어로 검색 후 성공률 확인

</details>

---

## 🔹 Part 2: HyDE - 가상 답변 생성 (20분)

### 📋 케이스: 파스텔 호텔 객실 정보

**질문**: "파스텔 호텔 객실 정보 알려줘"

**목표**: 호텔 정보 찾기
- 🏨 **호텔** 문서를 찾아야 합니다 (카페/극장 아님)

**관련 문서**:

In [ ]:
# HyDE 케이스 - 호텔 정보 (문서 10개)
hyde_docs_raw = [
    # 정답 문서 (파스텔 호텔 객실 50개)
    {"content": "파스텔 호텔: 서울 강남구 위치, 객실 50개, 주말 특가 이벤트. 체크인 14시, 체크아웃 11시. 조식 포함. 무료 와이파이, 피트니스 센터.", "type": "hotel"},
    # Noise 문서 9개 (파스텔 + 객실 언급으로 혼동)
    {"content": "블루문 호텔: 제주도 서귀포 위치, 객실 80개, 오션뷰, 수영장, 스파. 가족 단위 여행객에게 인기.", "type": "other_hotel"},
    {"content": "파스텔 브랜드 소개: 호텔, 카페, 소극장을 운영하는 복합 문화 공간. 객실 수는 각 지점마다 다름.", "type": "brand"},
    {"content": "파스텔 카페: 제주 애월 위치, 좌석 50개, 오션뷰. 디저트와 음료 판매.", "type": "cafe"},
    {"content": "호텔 객실 정보 확인 방법: 홈페이지 접속 후 객실 현황 조회, 전화 문의 가능.", "type": "guide"},
    {"content": "강남구 숙박 시설: 다양한 호텔, 게스트하우스, 에어비앤비 옵션 제공. 객실 규모 다양.", "type": "area"},
    {"content": "파스텔 이용 후기: 객실이 깨끗하고 직원 친절. 추천합니다.", "type": "review"},
    {"content": "호텔 객실 예약 팁: 체크인 시간 확인, 객실 타입 선택, 조식 포함 여부 확인.", "type": "tips"},
    {"content": "파스텔 소극장: 부산 해운대 위치, 좌석 100석, 토요일 공연.", "type": "theater"},
    {"content": "객실 수 비교: 파스텔 호텔 50개, 블루문 호텔 80개, 스카이 호텔 120개.", "type": "comparison"},
]

hyde_docs = [Document(page_content=d["content"], metadata={"type": d["type"]}) for d in hyde_docs_raw]
hyde_vectorstore = FAISS.from_documents(hyde_docs, embeddings)

print(f"📚 HyDE 문서 {len(hyde_docs_raw)}개 로딩:")
print("  정답: hotel (파스텔 호텔 객실 50개)")
print("  Noise: 9개 (파스텔/객실 키워드 포함으로 혼동 유발)")

In [ ]:
# HyDE 데모 실행
hyde_query = "파스텔 객실 몇개임?"

# 1) Naive RAG (비교용)
print("🔍 Naive RAG (구어체 질문):")
naive_results = hyde_vectorstore.similarity_search(hyde_query, k=1)
print(f"   찾은 문서: {naive_results[0].metadata['type']}")
print(f"   내용: {naive_results[0].page_content[:50]}...")

print("\n" + "="*50)

# 2) HyDE (정답 예시)
print("\n🚀 HyDE (가상 답변 생성):")

# 전문가가 작성한 가상 답변
hypothetical_answer = """파스텔 호텔은 서울 강남구에 위치하고 있습니다. 
객실은 총 50개이며, 체크인은 오후 2시, 체크아웃은 오전 11시입니다.
무료 조식이 포함되어 있으며, 무료 와이파이와 피트니스 센터를 이용하실 수 있습니다."""

print(f"   생성된 가상 답변: {hypothetical_answer[:80]}...")

# 가상 답변으로 검색
hyde_results = hyde_vectorstore.similarity_search(hypothetical_answer, k=1)
print(f"\n   찾은 문서: {hyde_results[0].metadata['type']}")
if hyde_results[0].metadata['type'] == 'hotel':
    print("   ✅ 성공! 파스텔 호텔 문서를 찾았습니다!")

### 📌 언제 사용하나요?

질문이 짧고 애매할 때, 답변을 먼저 상상해서 검색하면 더 정확합니다:

**✅ 사용하기 좋은 경우:**
- "파스텔 객실 몇개임?" ← 구어체, 짧음
- "호텔 정보" ← 너무 간결함
- "체크인 시간은?" ← 정보가 부족함

**❌ 효과가 적은 경우:**
- "파스텔 호텔의 객실 수, 위치, 시설 정보를 자세히 알려주세요" ← 이미 충분히 상세함
- "서울 강남구 파스텔 호텔 50개 객실..." ← 이미 키워드가 풍부함

### 🎯 핵심 아이디어

짧은 질문 → **가상의 상세한 답변** 생성 → 답변으로 검색
- "파스텔 객실?" → "파스텔 호텔은 서울 강남구에 위치하고 있습니다. 객실은 50개이며..." 생성
- 이 가상 답변으로 검색하면 실제 호텔 문서와 유사도가 높아짐

---

### 💡 실습 2: 가상 답변 작성하기

**미션**: 프롬프트를 작성해서 호텔 정보를 찾도록 만들기!

**요구사항**: 구어체 질문에 대한 상세한 가상 답변을 생성하세요

In [ ]:
# 여기를 수정하세요! 👇
# TODO: Few-shot 프롬프트를 작성해서 구어체 질문을 상세한 호텔 안내문으로 변환하세요

my_query = "파스텔 객실 몇개임?"

# Few-shot 프롬프트 작성
prompt = """"""

# LLM 호출
response = llm.invoke(prompt.format(query=my_query))

# 응답 파싱
if "답변:" in response:
    hypothetical_answer = response.split("답변:")[-1].strip()
else:
    hypothetical_answer = response.strip()

print(f"생성된 가상 답변: {hypothetical_answer[:100]}...")

# 가상 답변으로 실제 검색
results = hyde_vectorstore.similarity_search(hypothetical_answer, k=1)
found_type = results[0].metadata['type']

print(f"찾은 문서: {found_type} {'✅' if found_type == 'hotel' else '❌'}")

<details>
<summary>📝 정답 코드 보기 (클릭)</summary>

```python
# Few-shot 프롬프트
prompt = """당신은 호텔 정보를 정확하게 안내하는 직원입니다.

[예시]
질문: 신라호텔 객실 몇개임?
답변: 신라호텔은 서울 중구에 위치하고 있습니다. 객실은 464개이며, 체크인은 15시, 체크아웃은 12시입니다. 무료 조식과 수영장, 스파 시설을 이용하실 수 있습니다.

[실제 작업]
질문: {query}
답변:"""

# LLM 호출
response = llm.invoke(prompt.format(query=my_query))

# 응답 파싱
if "답변:" in response:
    hypothetical_answer = response.split("답변:")[-1].strip()
else:
    hypothetical_answer = response.strip()

print(f"생성된 가상 답변: {hypothetical_answer[:100]}...")

# 가상 답변으로 실제 검색
results = hyde_vectorstore.similarity_search(hypothetical_answer, k=1)
found_type = results[0].metadata['type']

print(f"찾은 문서: {found_type} {'✅' if found_type == 'hotel' else '❌'}")
```

**핵심 포인트:**
- 구어체 질문 → 상세한 호텔 안내문 형식으로 변환
- 위치, 객실 수, 시설 등 풍부한 키워드 포함
- 가상 답변으로 검색하면 실제 호텔 문서와 매칭 잘됨

</details>

---

## 🔹 Part 3: Step-Back - 추상화 (15분)

### 📋 케이스: 주말 체크인 변경 규정

**질문**: "파스텔 호텔 주말 체크인 변경 규정은?"

**목표**: 호텔 정책 문서 찾기
- 📋 **정책** 문서를 찾아야 합니다 (객실 정보 아님)

**관련 문서**:

In [ ]:
# Step-Back 케이스 - 호텔 정책 (문서 10개)
sb_docs_raw = [
    # 정답 문서 (일반적인 내용)
    {"content": "파스텔 호텔 이용 규정: 예약 변경 및 취소 정책, 환불 규정, 체크인/체크아웃 규정 안내.", "type": "policy"},
    # Noise 문서 9개 (구체적 키워드 많이 포함)
    {"content": "파스텔 호텔 객실: 50개, 체크인 14시, 체크아웃 11시. 조식 포함, 무료 주차.", "type": "room_info"},
    {"content": "토요일 체크인 혼잡 안내: 토요일 오전 체크인 시 대기 시간 30분 예상. 온라인 체크인 권장.", "type": "saturday_info"},
    {"content": "체크인 시간 연장 서비스: 12시까지 체크인 연장 가능. 추가 비용 5,000원.", "type": "late_checkin"},
    {"content": "파스텔 호텔 토요일 이벤트: 토요일 숙박 시 조식 무료 업그레이드.", "type": "saturday_event"},
    {"content": "오전 11시 체크인 안내: 정상 체크인은 14시. 11시 얼리 체크인은 사전 문의 필요.", "type": "early_checkin"},
    {"content": "주말 체크인 팁: 토요일-일요일 체크인은 혼잡. 12시 이전 도착 권장.", "type": "weekend_tip"},
    {"content": "체크인 시간 변경 방법: 마이페이지에서 체크인 시간 변경 가능. 당일 변경 불가.", "type": "change_method"},
    {"content": "파스텔 호텔 비용 안내: 객실 요금, 추가 서비스 비용, 연장 요금 안내.", "type": "cost_info"},
    {"content": "토요일 12시 체크인 가능 여부: 사전 요청 시 가능. 객실 상황에 따라 다름.", "type": "noon_checkin"},
]

sb_docs = [Document(page_content=d["content"], metadata={"type": d["type"]}) for d in sb_docs_raw]
sb_vectorstore = FAISS.from_documents(sb_docs, embeddings)

print(f"📚 Step-Back 문서 {len(sb_docs_raw)}개 로딩:")
print("  정답: policy (일반적인 이용 규정)")
print("  Noise: 9개 (구체적 키워드로 혼동 유발)")

In [ ]:
# Step-Back 데모 실행
sb_query = "파스텔 호텔 토요일 오전 11시 체크인을 12시로 늦추려면 비용이 얼마나 드나요?"

# 1) Naive RAG (비교용)
print("🔍 Naive RAG (매우 구체적인 질문):")
naive_results = sb_vectorstore.similarity_search(sb_query, k=1)
print(f"   찾은 문서: {naive_results[0].metadata['type']}")
print(f"   내용: {naive_results[0].page_content[:50]}...")

print("\n" + "="*50)

# 2) Step-Back (정답 예시)
print("\n🚀 Step-Back (추상화):")

# 전문가가 추상화한 질문
abstract_query = "파스텔 호텔 이용 규정"

print(f"   추상화된 질문: {abstract_query}")
print("   (구체적: '토요일 11시→12시 비용' → 추상적: '이용 규정')")

# 추상화된 질문으로 검색
sb_results = sb_vectorstore.similarity_search(abstract_query, k=1)
print(f"\n   찾은 문서: {sb_results[0].metadata['type']}")
if sb_results[0].metadata['type'] == 'policy':
    print("   ✅ 성공! 정책 문서를 찾았습니다!")

### 📌 언제 사용하나요?

질문이 너무 구체적일 때, 일반적인 주제로 추상화하면 더 넓은 문서를 찾을 수 있습니다:

**✅ 사용하기 좋은 경우:**
- "토요일 오전 11시 체크인을 12시로 늦추려면 비용은?" ← 너무 구체적 (시간, 요일)
- "일요일 3시 객실 예약 취소하면 수수료는?" ← 세부 정보 많음
- "금요일 저녁 7시 조식 메뉴는?" ← 특정 시간/요일

**❌ 효과가 적은 경우:**
- "호텔 이용 규정" ← 이미 충분히 일반적
- "예약 정책 알려줘" ← 추상화 불필요

### 🎯 핵심 아이디어

구체적 질문 → **일반적인 주제**로 추상화
- "토요일 11시 체크인 변경 비용" → "파스텔 호텔 예약 변경 비용 안내"
- 시간/요일 같은 구체적 정보 제거
- 일반적인 정책 문서를 찾음

---

### 💡 실습 3: 추상화하기

**미션**: 프롬프트를 작성해서 정책 문서를 찾도록 만들기!

**요구사항**: 구체적 질문을 일반적인 주제로 추상화하세요 (호텔 이름은 유지)

In [ ]:
# 여기를 수정하세요! 👇
# TODO: Few-shot 프롬프트를 작성해서 구체적인 질문을 일반적인 주제로 추상화하세요

my_query = "파스텔 호텔 토요일 오전 11시 체크인을 12시로 늦추려면 비용이 얼마나 드나요?"

# Few-shot 프롬프트 작성
prompt = """"""

# LLM 호출
response = llm.invoke(prompt.format(query=my_query))

# 응답 파싱
if "출력:" in response:
    abstract_query = response.split("출력:")[-1].strip()
else:
    abstract_query = response.strip()

abstract_query = abstract_query.split('\n')[0].strip()
print(f"추상화된 질문: {abstract_query}")

# 추상화된 질문으로 실제 검색
results = sb_vectorstore.similarity_search(abstract_query, k=1)
found_type = results[0].metadata['type']

print(f"찾은 문서: {found_type} {'✅' if found_type == 'policy' else '❌'}")

<details>
<summary>📝 정답 코드 보기 (클릭)</summary>

```python
# Few-shot 프롬프트
prompt = """당신은 구체적인 질문을 일반적인 주제로 추상화하는 전문가입니다.

[예시 1]
입력: 신라호텔 토요일 아침 7시 조식 메뉴는?
출력: 신라호텔 이용 안내

[예시 2]
입력: 롯데호텔 일요일 오후 3시 객실 예약 취소하면 수수료는?
출력: 롯데호텔 이용 규정

[실제 작업]
입력: {query}

규칙:
1. 호텔 이름은 반드시 유지하세요 (파스텔 호텔)
2. 구체적인 단어만 제거하세요 (토요일, 11시, 12시, 비용)
3. "이용 규정" 또는 "이용 안내"로 바꾸세요

출력:"""

# LLM 호출
response = llm.invoke(prompt.format(query=my_query))

# 응답 파싱
if "출력:" in response:
    abstract_query = response.split("출력:")[-1].strip()
else:
    abstract_query = response.strip()

abstract_query = abstract_query.split('\n')[0].strip()
print(f"추상화된 질문: {abstract_query}")

# 추상화된 질문으로 실제 검색
results = sb_vectorstore.similarity_search(abstract_query, k=1)
found_type = results[0].metadata['type']

print(f"찾은 문서: {found_type} {'✅' if found_type == 'policy' else '❌'}")
```

**핵심 포인트:**
- 구체적 질문 → 일반적 주제로 추상화
- 호텔 이름은 유지, 시간/요일/금액 제거
- "이용 규정", "이용 안내" 같은 일반 용어 사용
- 일반적인 정책 문서를 찾는 데 효과적

</details>

---

## 📊 Part 4: 3가지 기법 종합 비교

### 🎯 언제 어떤 기법을 쓸까?

| 기법 | 언제 사용? | 예시 |
|------|-----------|------|
| **Multi-Query** | 복합적인 질문 | "호텔 숙박하고 카페 가고 공연 보고 싶어요" |
| **HyDE** | 짧고 애매한 질문 | "파스텔 호텔 객실 정보" |
| **Step-Back** | 너무 구체적인 질문 | "주말 체크인 변경 규정" → "체크인 정책" |

### 💡 조합 전략

1. **먼저 Multi-Query로 분해**
2. **각 sub-query에 HyDE나 Step-Back 적용**
3. **모든 결과 합쳐서 재순위화** (다음 시간에!)

---

## 🎯 Part 5: 나만의 질문으로 실험하기

**미션**: 여러분만의 질문과 문서로 3가지 기법을 테스트해보세요!

In [ ]:
# 여기를 자유롭게 수정하세요! 👇

# 1) 나만의 문서 만들기
my_docs_raw = [
    {"content": "여기에 문서 내용을 입력하세요", "type": "doc1"},
    {"content": "여기에 문서 내용을 입력하세요", "type": "doc2"},
]

my_docs = [Document(page_content=d["content"], metadata={"type": d["type"]}) for d in my_docs_raw]
my_vectorstore = FAISS.from_documents(my_docs, embeddings)

# 2) 나만의 질문
my_query = "여기에 질문을 입력하세요"

# 3) 원하는 기법 선택 (Multi-Query, HyDE, Step-Back)
# ...

print("🎉 실험을 시작하세요!")

---

## 🎉 04번 완료!

### ✅ 배운 내용

1. **Multi-Query**: 복합 질문 분해
2. **HyDE**: 가상 답변 생성
3. **Step-Back**: 추상화

### 🚀 다음 시간 예고

**05번: 하이브리드 검색 & 재순위화**
- BM25 키워드 검색
- Vector + BM25 결합 (RRF)
- Cross-encoder 재순위화